# Tutorial Example
Go through synthesizAR example step-by-step from the beginning. Ideally this will end up in a tutorial in the docs.

In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
import astropy.constants as const
from astropy.coordinates import SkyCoord
from sunpy.sun import constants
from sunpy.map import Map
from sunpy.net import Fido,attrs as a
from sunpy.time import TimeRange

import synthesizAR
import synthesizAR.extrapolate
from synthesizAR.util import heeq_to_hcc_coord

%matplotlib inline

/Users/willbarnes/anaconda/envs/synthesizar/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Download Magnetogram Data

## Prep Magnetogram

## Extrapolate Field and Trace Fieldlines

## Create Active Region

## Run Hydrodynamic Simulation

### Setup A Heating Model

In [ ]:
class OutofPhaseNanoflares(object):
    def __init__(self,heating_options):
        self.heating_options = heating_options
        
    def calculate_event_properties(self,loop):
        self.number_events = 1
        start_time = np.random.uniform(low=0, high=self.base_config['total_time'] - self.heating_options['duration'])
        max_energy = (self.heating_options['stress_level'] * loop.field_strength.mean().value)**2/(8.*np.pi)
        return {'magnitude': np.array([max_energy/(self.heating_options['duration']/2.)]),
                'rise_start': np.array([start_time]),
                'rise_end': np.array([start_time+self.heating_options['duration']/2]),
                'decay_start': np.array([start_time+self.heating_options['duration']/2]),
                'decay_end': np.array([start_time+self.heating_options['duration']])}

In [ ]:
heating_options = {
    'duration':200.0,
    'stress_level':0.3,
}
heating_model = OutofPhaseNanoflares(heating_options)

### Configure and Run Simulation

In [ ]:
ih = synthesizAR.util.InputHandler(
    '/storage-home/w/wtb2/codes/ebtelPlusPlus/config/ebtel.example.cfg.xml')
base_config = ih.lookup_vars()
base_config['c1_cond0'] = 6.0
base_config['total_time'] = 3e4
base_config['use_adaptive_solver'] = True
base_config['use_flux_limiting'] = True
base_config['calculate_dem'] = False
base_config['heating']['partition'] = 1.0
base_config['heating']['background'] = 1e-6
base_config['force_single_fluid'] = False
base_config['tau_max'] = 200.0

In [ ]:
ebtel_interface = EbtelInterface(
    base_config,heating_model,
    '/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling_outofphase_long/hydro_config/',
    '/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling_outofphase_long/hydro_results/')

In [ ]:
field.configure_loop_simulations(ebtel_interface,)

In [ ]:
def ebtel_runner(loop):
    subprocess.call([os.path.join('/storage-home/w/wtb2/codes/','ebtelPlusPlus/bin/ebtel++.run'),
                     '-c',loop.hydro_configuration['config_filename']])

In [ ]:
field.load_loop_simulations(
    ebtel_interface,
    '/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling_outofphase_long/loop_parameters.h5')

Plot the output of all of our loop simulations

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(20,10),sharex=True)
plt.subplots_adjust(hspace=0.)
for loop in field.loops[::5]:
    axes[0].plot(loop.time,loop.electron_temperature[:,0].to(u.MK),color='C0',alpha=0.1)
    axes[0].plot(loop.time,loop.ion_temperature[:,0].to(u.MK),color='C2',alpha=0.1)
    axes[1].plot(loop.time,loop.density[:,0]/1e9,color='C0',alpha=0.05)
axes[0].set_xlim([0,base_config['total_time']])
axes[0].set_ylabel(r'$T$ [MK]')
axes[1].set_ylabel(r'$n$ [10$^9$ cm$^{-3}$]')
axes[1].set_xlabel(r'$t$ [s]')